In [118]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import os
import re
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

In [119]:
df = pd.read_parquet('')
df.head(2)

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,NOME_RESPONSAVEL,treated_text,IOB
0,None,214/2020,Aquisição de cabos elétricos especiais e ótico...,PREGÃO ELETRÔNICO,092.04570/2020,10/11/2020,None,974200,www.comprasnet.gov.br,None,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO PE Nº 214...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,None,208/2020,Aquisição de conjuntos motobomba do tipo subme...,PREGÃO ELETRÔNICO,092.017888/2020,10/11/2020,None,974200,www.comprasnet.gov.br,None,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO PE Nº 208...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...


In [120]:
from nltk.tokenize import word_tokenize

In [121]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == word_tokenize(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in word_tokenize(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(word_tokenize(row[entity])) and \
                    token == word_tokenize(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(word_tokenize(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [122]:
iob = IOBifyer(column='treated_text')
r = iob.transform(df)

In [123]:
r

0     [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
1     [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
2     [O, O, O, O, O, O, O, B-MODALIDADE_LICITACAO, ...
3     [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
4     [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
                            ...                        
61    [O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORGA...
62    [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
63    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
64    [O, O, O, O, O, B-MODALIDADE_LICITACAO, I-MODA...
65    [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
Length: 66, dtype: object

In [124]:
df.shape, r.shape

((66, 12), (66,))

In [125]:
df["IOB"] = np.nan

In [126]:
for i in range(len(df)):
    df.loc[i, "IOB"] = ' '.join(r[i])

In [127]:
df['treated_text'][0]

'AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO PE Nº 214/2020 Processo: 092.04570/2020. OBJETO: Aquisição de cabos elétricos especiais e óticos para uso nos sistemas de automação do Sistema de Abastecimento de Água (SAA) e do Sistema de Esgotamento Sanitário (SES) da Caesb, para possibilitar a reposição nos referidos equipamentos industriais. CRITÉRIO DE JULGAMENTO: Menor Preço; DOTAÇÃO ORÇAMENTÁRIA: UO: 22202; PROGRAMA DE TRABALHO: 17.122.8209.8517/6977; NATUREZA DE DESPESA: 33.90.30; Código de Aplicação: 12.203.205.200-07. FONTE DE RECURSO: Recursos Próprios, CÓDIGO: 11.101.000.000-3. ENTREGA: 90 dias. ABERTURA: 10/11/2020, às 09 horas no site www.comprasnet.gov.br (UASG: 974200). INFORMAÇÕES: O edital e seus anexos encontram-se disponíveis nos sites: www.caesb.df.gov.br e www.comprasnet.gov.br — menu Licitações, a partir do dia 26/10/2020. Fone: (61) 3213-7122, E-mail: licitacao Ocaesb.df gov.br. ELISA TEREZINHA HAMMES Pregoeira'

In [128]:
len(word_tokenize(df['treated_text'][0])), len(df['IOB'][0].split())

(163, 163)

In [129]:
df['IOB'][0]

'O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO O O B-NUM_LICITACAO O O B-PROCESSO O O O B-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-DATA_ABERTURA O O O O O O B-SISTEMA_COMPRAS O O O B-CODIGO_SISTEMA_COMPRAS O O O O B-IOB O O O O O O O O O O O B-SIS

In [130]:
for i in range(len(df)):
    if len(word_tokenize(df['treated_text'][i])) != len(df['IOB'][i].split()):
        print(i)